In [42]:
import pandas as pd
import json

cdf = pd.read_csv('census_data.csv',index_col='state', encoding='utf-8') #census data
hdf = pd.read_csv('health.csv', encoding='utf-8') #health data from the cdc
#print(df.corr()['depression'])

In [72]:
cdf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 52 entries, Alabama to Guam
Data columns (total 39 columns):
year                                                                                                                                                                                       52 non-null int64
locationAbbr                                                                                                                                                                               52 non-null object
depression                                                                                                                                                                                 52 non-null float64
estimateTotalHouseholds                                                                                                                                                                    52 non-null int64
medianIncomeAll                                                        

In [111]:
hdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8957 entries, 0 to 8956
Data columns (total 27 columns):
Year                          8957 non-null int64
Locationabbr                  8957 non-null object
Locationdesc                  8957 non-null object
Class                         8957 non-null object
Topic                         8957 non-null object
Question                      8957 non-null object
Response                      8957 non-null object
Break_Out                     8957 non-null object
Break_Out_Category            8957 non-null object
Sample_Size                   8957 non-null object
Data_value                    8940 non-null float64
Confidence_limit_Low          8940 non-null float64
Confidence_limit_High         8940 non-null float64
Display_order                 8957 non-null object
Data_value_unit               8957 non-null object
Data_value_type               8957 non-null object
Data_Value_Footnote_Symbol    17 non-null object
Data_Value_Footnote       

In [68]:
correlation_results = []
results_dict = {}

for i in hdf.Question.unique():
    #break down into specific questions
    q_df = hdf[hdf['Question'] == i]
    
    for a in q_df.Response.unique():
        #break down into specific responses for each question
        r_df = q_df[q_df['Response'] == a].set_index('Locationdesc')
        # rename data_values to the question and response 
        question = str(r_df['Question'].unique()[0])
        response = str(a)

        name = question + "__" + response
        
        #array of values connected to the above question and response
        cor_query = r_df["Data_value"].rename(name)
        #print(cor_query)
        # append the data_value of this particular questions and response by state to the census data
        
        try:
            #print("Mining correlations for....")
            #print(name)    
            correlations = pd.concat([cdf, cor_query], axis=1).corr()[name]
            #print("correlations \n")
            indexer = 0
            for index, value in correlations.iteritems():
                if value > .65 and value < 1:
                    print(correlations.name)
                    print(value)
                    print(index)
                    
                    results_dict[correlations.name] = {
                            'correlation':value,
                            "values":[cor_query.values],
                            index:[cdf[index].values],
                            'states': cdf.index.values,
                            }
                    
                    
                    correlation_results.append(
                        {correlations.name:[{
                            'correlation':value,
                            "values":[cor_query.values],
                            index:[cdf[index].values],
                            'states': cdf.index.values,
                            }
                        ]})
                    
                    indexer += 1
                    
                    print("---" * 20)
                elif value < -.65:
                    print(correlations.name)
                    print(value)
                    print(index)
                    print("---" * 20)
                    
                    results_dict[correlations.name] = {
                            'correlation':value,
                            "values":[cor_query.values],
                            index:[cdf[index].values],
                            'states': cdf.index.values,
                            }
                    
                    correlation_results.append(
                        {correlations.name:[{
                            'correlation':value,
                            correlations.name + "_values":[cor_query.values],
                            index:[cdf[index].values],
                            'states': cdf.index.values,
                            }
                        ]})
                    
                
            
        except Exception as e:
            print(e)
            print(f"cannot process {name}")



What is your race/ethnicity?__Black
0.849684601251
Total; Estimate; FAMILIES - Families - Female householder, no husband present
------------------------------------------------------------
What is your race/ethnicity?__White
0.715366725712
depression
------------------------------------------------------------
What is your race/ethnicity?__White
-0.709440116955
Total; Estimate; FAMILIES - Families - Female householder, no husband present
------------------------------------------------------------
What is your race/ethnicity?__Other
-0.681008787242
depression
------------------------------------------------------------
Adults who have been told they have arthritis (variable calculated from one or more BRFSS questions)__Yes
0.737767568264
depression
------------------------------------------------------------
Adults who have been told they have arthritis (variable calculated from one or more BRFSS questions)__No
-0.738537862417
depression
-----------------------------------------------

Was there a time in the past 12 months when you needed to see a doctor but could not because of cost?__Yes
-0.703514775976
medianIncomeAll
------------------------------------------------------------
Was there a time in the past 12 months when you needed to see a doctor but could not because of cost?__Yes
-0.76613368181
Median income (dollars); Estimate; FAMILIES - Families
------------------------------------------------------------
Was there a time in the past 12 months when you needed to see a doctor but could not because of cost?__Yes
-0.775299029672
Median income (dollars); Estimate; FAMILIES - Families - With own children under 18 years
------------------------------------------------------------
Was there a time in the past 12 months when you needed to see a doctor but could not because of cost?__Yes
-0.657655764892
Male; Estimate; MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2014 INFLATION-ADJUSTED DOLLARS) - Population 25 years and over with earnings
-----------------------------

In [83]:
results_dict["Was there a time in the past 12 months when you needed to see a doctor but could not because of cost?__Yes"]

{'Total; Estimate; MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2014 INFLATION-ADJUSTED DOLLARS) - Population 25 years and over with earnings - High school graduate (includes equivalency)': [array([25693, 35963, 27120, 25749, 27664, 30568, 32916, 31910, 28125,
         25206, 26197, 30509, 24752, 28847, 29174, 29741, 28063, 26201,
         27446, 26240, 32369, 32137, 26372, 30858, 26236, 27080, 25751,
         28617, 29608, 31871, 31921, 25086, 29501, 26101, 32599, 28402,
         27380, 26075, 30098, 30605, 25086, 27443, 25517, 27038, 29498,
         29753, 29126, 31016, 26671, 30102, 30662, 14524], dtype=int64)],
 'correlation': -0.71714025329816644,
 'states': array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
        'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
        'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
        'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
        'Massachusetts', 'Michigan', 'Minnesota', '

In [117]:
# columns from the Census data we want to pull for graphing
blindness = pd.Series(results_dict["Are you blind or do you have serious difficulty seeing, even when wearing glasses?__Yes"]['values'][0]).rename('blindness')
depression = pd.Series(cdf['depression'].values).rename('percentDepressed')
arthritis = pd.Series(results_dict["Adults who have been told they have arthritis (variable calculated from one or more BRFSS questions)__Yes"]['values'][0]).rename('Arthritis')

poverty_line = pd.Series(cdf['Total Civilian Noninstitutionalized Population; Estimate; 100 to 149 percent of the poverty level'].values).rename("povertyPop")
state_abbr = pd.Series(cdf["locationAbbr"].values).rename('stateAbbr')
states = pd.Series(results_dict["Are you blind or do you have serious difficulty seeing, even when wearing glasses?__Yes"]['states']).rename('states')
medianIncome = pd.Series(cdf['medianIncomeAll'].values).rename('medianIncome')
doc_too_expensive = pd.Series(results_dict["Was there a time in the past 12 months when you needed to see a doctor but could not because of cost?__Yes"]['values'][0]).rename('healthcare_unaffordable')
long_term_unemployment = pd.Series(results_dict["What is your employment status?__No work >yr"]["values"][0]).rename('unemployment')

In [118]:
df = pd.concat(
        [
                state_abbr,
                states,
                poverty_line,
                blindness,
                depression, 
                arthritis,
                medianIncome,
                doc_too_expensive,
                long_term_unemployment],
    
        axis=1).set_index('stateAbbr').dropna()

In [119]:
df

,states,povertyPop,blindness,percentDepressed,Arthritis,medianIncome,healthcare_unaffordable,unemployment
stateAbbr,,,,,,,,
AL,Alabama,11.1,6.6,21.3,34.0,42830.0,17.4,3.2
AK,Alaska,7.7,2.7,15.6,21.9,71583.0,11.9,2.7
AZ,Arizona,10.3,4.9,18.7,24.9,50068.0,15.5,3.1
AR,Arkansas,12.3,6.8,21.5,30.4,41262.0,18.2,3.7
CA,California,10.2,5.0,13.3,20.4,61933.0,13.5,4.0
CO,Colorado,8.3,3.0,17.0,22.8,61303.0,13.1,2.4
CT,Connecticut,6.3,3.3,18.3,24.0,70048.0,11.3,3.4
DE,Delaware,7.0,3.8,18.3,26.6,59716.0,10.8,2.7
DC,District of Columbia,11.0,6.1,18.1,19.2,71648.0,10.5,4.1


In [121]:
df.to_csv('data.csv', encoding='utf-8')